**Задание на зачет:**

У вас есть документ: Info.xlsx

Вам нужно на python написать скрипт, с помощью которого генерируется 1000 разных pdf файлов по строкам из Info.xlsx.

Вид итогового документа:

![](example.png)

Способ формирования документа:

Из xlsx файла убирается случайным образом от (0; 30] % строк. 

* Одна строка – это набор значений из колонок A-G из исходного xlsx документа (набор значений не меняется) 
* +3 дополнительных сгенеренных значения для колонок **'Ориентировочный объем', 'Стоимость за единицу', 'Всего'**:

**1. Ориентировочный объем** – это случайное целое значение в интервале от 1 до 100

**2. Стоимость** – случайное числовое значение [0 ; 1000000], округление до 2-го знака (до копейки). В итоговом документе должно быть [10 ; 40]% нулевых значений, 50% значений: (0 ; 100000], остальные – со значениями больше 100000

**3. Всего** – произведение объема и стоимости.

Convert Excel to PDF in Python | XLSX to PDF, XLS to PDF in Python

https://blog.aspose.com/2021/04/02/convert-excel-files-to-pdf-in-python/

In [1]:
#!pip install xlsxwriter
#!pip install aspose-cells
#!pip install PyPDF2
#!pip install pymupdf

In [2]:
import pandas as pd
import numpy as np
import random
import os

import jpype
import asposecells
jpype.startJVM()
from asposecells.api import Workbook, SaveFormat, PdfSaveOptions, PageOrientationType

https://pythonru.com/uroki/osnovy-pandas-3-vazhnye-metody-formatirovanija-dannyh#:~:text=%D0%A1%D0%BF%D0%BE%D1%81%D0%BE%D0%B1%D1%8B%20%D0%BE%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F%3A%20inner%2C%20outer%2C%20left%2C%20right&text=%D0%92%20SQL%20%D0%B5%D1%81%D1%82%D1%8C%204%20%D1%82%D0%B8%D0%BF%D0%B0,%D0%BC%D0%BE%D0%B6%D0%BD%D0%BE%20%D0%BD%D0%B0%D0%B9%D1%82%D0%B8%20%D0%B2%20%D0%BE%D0%B1%D0%B5%D0%B8%D1%85%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%D1%85.

### 1. Загрузка

In [3]:
cols = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] #, 'K', 'L']
head = pd.read_excel('Info.xlsx', names=cols)[:6]
head['I'][4]  = 'Стоимость за единицу, руб'
head['J'][4]  = 'Итого стоимость, руб'
#head.pop('L')
#head.pop('K')
#head

In [4]:
# Load the xlsx file
data = pd.read_excel('Info.xlsx', header=6,  usecols = list(range(9)), names=cols) # index_col = list(range(5)),
#data.head(3)

In [5]:
n,m = np.shape(data)
n,m
data_end = 0
for i in range(n-1,1,-1):
    if data['I'][i] == 'ИТОГО':
        data_end = i
footer = data[:][data_end:]
data = data[:][:data_end]

In [6]:
#df

In [7]:
#footer

### 2. Предобработка df

In [8]:
# def GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total):
#     dct = {
#         'num':num,              #0   № п/п   
#         'name':name,            #1   Наименование обращение                
#         'status':status,        #2   Признак (статус) Заявки (обращения)  
#         'damaged':damaged,      #3   Поврежденные элементы/поверхности (тип, материал, др.)
#         'jobs':jobs,            #4   Виды работ 
#         'unit':unit,            #5   Ед. изм.  
#         'serviceNum':serviceNum,#6   Номер услуги     
#         'volume':volume,        #7   Объем в год
#         'cost':cost,            #8   Стоимость
#         'total':total}          #11  Итого         
#     return dct

In [9]:
def GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total):
    dct = {
        'A':num, 
        'B':name,                  
        'C':status,      
        'D':damaged,     
        'E':jobs,          
        'F':unit,         
        'G':serviceNum,
        'H':volume,       
        'I':cost,           
        'J':total}     
    return dct

In [10]:
num = ''
name = ''
status = ''
damaged = ''
jobs = ''
unit = ''
serviceNum = ''
#volume = 0
#cost = 0
#total = 0

In [11]:
arr=[]
for i in range(0,data_end):
    if not(pd.isna(data['A'][i])):
        num = data['A'][i]
        name = data['B'][i]
        status = data['C'][i]
    if not(pd.isna(data['D'][i])):  
        damaged = data['D'][i]
    jobs = data['E'][i]
    unit = data['F'][i]
    serviceNum = str(int(data['G'][i]))
    dct = GetDct(num, name, status, damaged, jobs, unit, serviceNum, 0,0,0) # volume, cost, total)
    arr.append(dct)
df = pd.DataFrame(arr)
#df

### 3. Обработка

In [12]:
arr=[]
p = random.randint(0,30)
for i in range(0, data_end):
    if random.randint(0, 100) > p: 
        num = df['A'][i]
        name = df['B'][i]
        status = df['C'][i]
        damaged = df['D'][i]
        jobs = df['E'][i]
        unit = df['F'][i]
        serviceNum = str(int(df['G'][i]))
        volume = random.randint(1,100)
        p1 = random.randint(1,10)
        if p1 < 6:                     
            cost = round(random.uniform(0, 1e5), 2)
        elif p1 < random.randint(6,9): 
            cost = 0
        else:                      
            cost = round(random.uniform(1e5, 1e6), 2)
        total = volume * cost 
        dct = GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total)
        arr.append(dct)
newdf = pd.DataFrame(arr)
newdf.head(3)

,A,B,C,D,E,F,G,H,I,J
0,1,Мелкий ремонт пола (внутренняя отделка),Текущая Заявка,Бетонное основание,Демонтаж стяжки толщиной до 50 мм,кв.м.,1800000401,50,670985.68,33549284.00
1,1,Мелкий ремонт пола (внутренняя отделка),Текущая Заявка,Бетонное основание,Демонтаж армированной стяжки толщиной до 100 мм,кв.м.,1800000402,69,205120.97,14153346.93
2,1,Мелкий ремонт пола (внутренняя отделка),Текущая Заявка,Бетонное основание,Нанесение ремонтного состава на бетонную повер...,кв.м.,1800000403,69,163646.38,11291600.22


In [13]:
newdf[len(newdf)-100:len(newdf)-50]

,A,B,C,D,E,F,G,H,I,J
772,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Устройство тротуарной плитки толщиной до 50 мм...,кв.м.,1800001541,63,37938.50,2390125.50
773,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Устройство тротуарной плитки толщиной до 50 мм...,кв.м.,1800001542,74,56185.02,4157691.48
774,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Устройство тротуарной плитки толщиной до 50 мм...,кв.м.,1800001543,26,84433.58,2195273.08
775,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Демонтаж решеток ливневой канализации,пог.м,1800001544,98,973089.57,95362777.86
776,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Замена решеток ливневой канализации,пог.м,1800001546,13,29376.14,381889.82
777,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Демонтаж бетонных лотков ливневой канализации,пог.м,1800001548,20,37584.52,751690.40
778,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Монтаж бетонных лотков ливневой канализации (в...,пог.м,1800001549,13,20727.03,269451.39
779,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Демонтаж пластиковых лотков ливневой канализации,пог.м,1800001550,59,0.00,0.00
780,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Монтаж пластиковых лотков ливневой канализации...,пог.м,1800001551,85,0.00,0.00
781,10,Мелкий ремонт прилегающей территории,Текущая Заявка,"Дорожное покрытие, тротуар, отмостка",Демонтаж бордюрных камней с сохранением,пог.м,1800001552,13,758351.46,9858568.98


In [14]:
newdf['H'][0]

50

### 4. Проверка

**Количество строк в новом документе**

In [15]:
len_newdf = len(newdf)
len_newdf

872

**Процент строк от исходного документа**

In [16]:
len_newdf / data_end * 100

70.20933977455717

**Процент нулевых значений 'Стоимость за единицу'**

In [17]:
len(newdf[newdf.I == 0]) / len_newdf * 100

14.678899082568808

**Процент значений (0 , 100000] 'Стоимость за единицу'**

In [18]:
len(newdf[(newdf.I > 0) & (newdf.I <= 100000)]) / len_newdf * 100

51.49082568807339

### 5. Обратное преобразование документа

In [19]:
arr=[]

num = newdf['A'][0]
name = newdf['B'][0]
status = newdf['C'][0]
damaged = newdf['D'][0]
jobs = newdf['E'][0]
unit = newdf['F'][0]
serviceNum = newdf['G'][0]
volume = newdf['H'][0]
cost = newdf['I'][0]
total = newdf['J'][0]

dct = GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total)
arr.append(dct)

In [20]:
#arr

In [21]:
def GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total):
    dct = {
        'A':num, 
        'B':name,                  
        'C':status,      
        'D':damaged,     
        'E':jobs,          
        'F':unit,         
        'G':serviceNum,
        'H':volume,       
        'I':cost,           
        'J':total}  
    return dct

In [22]:
# for i in range(1,len_newdf):
#     if newdf['A'][i-1] == newdf['A'][i]:
#         num = np.nan
#         name = np.nan
#         status = np.nan
#     else:
#         num = newdf['A'][i]
#         name = newdf['B'][i]
#         status = newdf['C'][i]
        
#     if newdf['D'][i-1] == newdf['D'][i]:  
#         damaged = np.nan
#     else:    
#         damaged = newdf['D'][i]
#     jobs = newdf['E'][i]
#     unit = newdf['F'][i]
#     serviceNum = str(int(newdf['G'][i]))
#     dct = GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total)
#     arr.append(dct)
    
# df_for_docs = pd.DataFrame(arr)

In [23]:
# df_for_docs['A']

In [24]:
# df_for_docs[df_for_docs['A']==2.0]

### 6. Склеиваем обратно

In [25]:
#docs = pd.merge(head, df_for_docs, how = 'outer')

In [26]:
#docs[140:150]

In [27]:
#docs = pd.merge(docs, footer, how = 'outer')

In [28]:
#docs[1000:1020]

In [29]:
docs = pd.merge(head, newdf, how = 'outer')
docs = pd.merge(docs, footer, how = 'outer')

In [30]:
for col in ['A', 'B', 'C', 'D']:
    last_valid = docs.at[6, col]

    for i in range(10, len(docs) - 7):
        if pd.isna(docs.at[i, col]):
            docs.at[i, col] = last_valid
        else:
            last_valid = docs.at[i, col]

docs[len(docs)-50:]

,A,B,C,D,E,F,G,H,I,J
839,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Сверление отверстий диаметром от 25 до 50 мм в...,шт.,1800001596,25,0.0,0.0
840,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Сверление отверстий диаметром до 25 мм в метал...,шт.,1800001597,76,9879.11,750812.36
841,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Сверление отверстий диаметром от 25 до 50 мм в...,шт.,1800001598,4,500541.32,2002165.28
842,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Сверление отверстий диаметром от 25 до 50 мм в...,шт.,1800001601,76,5415.61,411586.36
843,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Сверление в бетонных и железобетонных конструк...,шт.,1800001602,56,0.0,0.0
844,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Покраска труб отопления масляной краской и эмалью,пог.м,1800001604,62,65475.47,4059479.14
845,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Порошковая окраска металлических поверхностей,пог.м,1800001606,71,60657.62,4306691.02
846,14,Вспомогательные работы,Текущая Заявка,Теплоизоляция,Укрывание защитной плёнкой,кв.м.,1800001607,10,0.0,0.0
847,15,Изготовление дубликатов ключей,Текущая Заявка,Теплоизоляция,Изготовление дубликата плоского одностороннего...,шт.,1800001608,100,48060.12,4806012.0
848,15,Изготовление дубликатов ключей,Текущая Заявка,Теплоизоляция,Изготовление дубликата плоского двухстороннего...,шт.,1800001609,30,784057.33,23521719.9


In [31]:
docs[:30]

,A,B,C,D,E,F,G,H,I,J
0,NaN,NaN,NaN,NaN,к Техническому заданию,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
2,NaN,NaN,ПЕРЕЧЕНЬ РАБОТ ПО МЕЛКОМУ РЕМОНТУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Признак (статус) Заявки (обращения),"Поврежденные элементы/поверхности (тип, матери...",Виды работ,Ед. изм.,Номер услуги,"Ориентировочный объем в год (кв.м, п.м., шт.)","Стоимость за 1 единицу (кв.м, п.м., шт.) без ...","Стоимость НДС1, руб.","Стоимость за единицу, руб","Итого стоимость, руб"
9,3,4,5,6,7,8,9,10,11,12


### 7. Формирование Excel файла

In [32]:
path = 'output'
#os.mkdir(path)

https://pythobyte.com/reading-and-writing-excel-files-in-python-with-the-pandas-library-8358adce/

In [33]:
#docs.loc[len(docs) - 9]

In [34]:
#docs['C'][len(docs)-50:len(docs)-7]

In [35]:
k=0
writer = pd.ExcelWriter(path='output\output{}.xlsx'.format(k))
docs.to_excel(excel_writer = writer, 
              sheet_name = 'main',
              header=False, 
              index=False, 
              #merge_cells = True #False
             )

def generate(start, end):
    return random.randint(start, end)
FONST = ['Times New Roman', 'Calibri', 'Aial']
font_name = FONST[generate(0, 2)]
font_size = generate(9, 12)

width = [54, 85, 62, 110, 221, 54, 74, 54, 54, 54, 54, 54]

workbook = writer.book
worksheet = writer.sheets['main']
worksheet.set_landscape()
worksheet.fit_to_pages(1, 0) 
worksheet.set_row(5, 80)

merge_format = workbook.add_format({'text_wrap': True, 
                                    'border': 1, 
                                    'align': 'center', 
                                    'valign': 'top'})

for col, col_ind in [('A',0), ('B',1), ('C',2), ('D',3)]:
    startCells = [10]
    for row in range(11, len(docs) - 8):
        if docs.loc[row, col] != docs.loc[row-1, col]:
            startCells.append(row)
    startCells.append(len(docs) - 7)

    for i in range(len(startCells)-1):
        startRow = startCells[i]
        endRow = startCells[i+1]-1;
        if startRow == endRow:
            s = docs.loc[startRow, col] #startRow-1 
            worksheet.write(startRow, col_ind, s , merge_format)
        else:
            s = docs.loc[startRow, col]
            worksheet.merge_range(startRow, col_ind, endRow, col_ind, s , merge_format)


merge_format.set_font_name(font_name)
merge_format.set_font_size(font_size)

for column, w in zip(docs, width):
    col_idx = docs.columns.get_loc(column)
    worksheet.set_column(col_idx, col_idx, w / 6, merge_format)
    
    
cell_format = workbook.add_format({'bg_color': '#deebf6', 
                                       'text_wrap': True, 
                                       'border': 2, 
                                       'bold': True,
                                       'font_name': font_name,
                                       'font_size': font_size,
                                       'align': 'center',
                                       'valign': 'vcenter'
                                       })


for i in range(8, 10):
    for j, col in zip(range(len(cols)), cols):
        s = docs.loc[i, col]
        worksheet.write(i, j, s, cell_format)

for i in range(4,7):
    s = docs.loc[len(docs) - i, 'A']
    worksheet.merge_range(len(docs) - i,  0, len(docs) - i, 3, s)

merge_format = workbook.add_format({'text_wrap': True, 
                                    #'border': 1, 
                                    'align': 'right', 
                                    'bold': True,
                                    'valign': 'top',
                                    'font_size': 15 })
worksheet.merge_range('A1:J1', 'Приложение '+str(i)+'\nк Техническому заданию', merge_format)
worksheet.merge_range('A3:J8', 'ПЕРЕЧЕНЬ ПО МЕЛКОМУ РЕМОНТУ', merge_format)
merge_format = workbook.add_format({'text_wrap': True, 
                                    #'border': 1, 
                                    'align': 'right', 
                                    #'bold': True,
                                    'valign': 'top'})
worksheet.merge_range('A2:J2', 'к Техническому заданию', merge_format)


writer.save()
#writer.close()

### 7. Формирование PDF файла

In [36]:
workbook = Workbook('output\output{}.xlsx'.format(k))
workbook.save('output\output{}.pdf'.format(k), SaveFormat.PDF)

**Задание на экзамен:**

Дополнительно нужно: 
1. Cлучайным образом поворачивать таблицу в документе, угол поворота: от -10 (вкл) до 10 (вкл) градусов. 
2. Менять случайным образом шрифт (выбирать один из вашего списка Font Name, на 0-2 единицы менять Font Size, и немного левее/правее сдвигать положение таблицы в документе.

In [37]:
from pathlib import Path
import fitz
import numpy as np
from PIL import Image

In [38]:
pdffile = 'output/output{}.pdf'.format(str(k))
doc = fitz.open(pdffile)
angle = generate(-10, 10) 

for page_id in range(doc.page_count):
    page = doc.load_page(page_id)
    pix = page.get_pixmap(alpha=False, dpi=154)
    im = np.frombuffer(buffer=pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
    im = Image.fromarray(im)
    im = im.crop((0, 50, pix.width, pix.height))

    im = im.rotate(angle)
    im.save('_tmp.pdf')

    page_front = fitz.open()
    page_front.insert_pdf(fitz.open('_tmp.pdf'), from_page=page_id, to_page=page_id)
    page.show_pdf_page(page.rect, page_front, pno=0, keep_proportion=True, overlay=True, oc=0, rotate=0, clip=None)

doc.save('output/pdf_rotated{}.pdf'.format(str(k), encryption=fitz.PDF_ENCRYPT_KEEP))